In [3]:
import sys
import os

sys.path.append("..")

In [5]:
import copy

import numpy as np
from gensim.models import FastText
from gensim.models.fasttext import save_facebook_model

from quantitizer.utils import mb
from quantitizer.integration.gensim.fasttext import make_new_fasttext_model

ft = FastText.load_fasttext_format("../data/cc.en.300.bin").wv
mb(ft)

/var/folders/lm/0j0v2m453ks57mmwdnjfvz700000gn/T/ipykernel_47221/163731476.py:10: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  ft = FastText.load_fasttext_format("../data/cc.en.300.bin").wv


7215.576073646545

In [13]:
# ft_float16 = make_new_fasttext_model(
#     ft, ft.vectors.astype(np.float16),
#     ft.vectors_ngrams.astype(np.float16))
# mb(ft_float16)

2533.0597734451294

In [6]:
import quantitizer as q
import importlib
importlib.reload(q)

ft_quantitized = make_new_fasttext_model(
    ft, q.quantitize(ft.vectors, sub_size=2, n_iter=5),
    q.quantitize(ft.vectors_ngrams, sub_size=2, n_iter=5))
mb(ft_quantitized)

268.9799814224243

In [19]:
ft_quantitized.vectors

PQ(
    vectors=2000000,
    dim=300,
    qdim=2,
    centroids=256,
    indexes=array([[ -29,  -53],
           [ 105,   76],
           [ -29,  -85],
           ...,
           [ -89, -128],
           [  59,   59],
           [ -68,  -68]], dtype=int8),
    codes=array([[[ 0.0991  , -0.05472 , -0.0632  , ..., -0.1407  , -0.00492 ,
              0.0662  ],
            [-0.013466,  0.000627,  0.0339  , ...,  0.1682  ,  0.0283  ,
             -0.02168 ],
            [ 0.126   , -0.05    , -0.03745 , ..., -0.0897  , -0.02342 ,
              0.12286 ],
            ...,
            [-0.01318 ,  0.0129  , -0.0374  , ..., -0.0334  ,  0.03912 ,
              0.003986],
            [-0.01445 , -0.01126 , -0.01611 , ..., -0.02122 , -0.003088,
             -0.000951],
            [ 0.0056  ,  0.02817 ,  0.005966, ...,  0.01335 , -0.001749,
              0.05124 ]],
    
           [[-0.0436  , -0.0416  , -0.07825 , ...,  0.1666  ,  0.01223 ,
              0.02135 ],
            [-0.005585, -0.0

In [5]:
ft_quantitized.save("out/compressed")

In [13]:
import temp
import importlib
importlib.reload(temp)

from gensim.models.fasttext import FastTextKeyedVectors

# ft_quantitized = temp.CompressedFastTextKeyedVectors.load("out/compressed")
ft_quantitized = temp.FastTextKeyedVectors.load("out/compressed")
# ft_quantitized = FastText.load("out/compressed")

In [14]:
ft_quantitized.vectors_ngrams

PQ(
    vectors=2000000,
    dim=300,
    qdim=2,
    centroids=256,
    indexes=array([[183,  29],
           [  7,  66],
           [208, 220],
           ...,
           [ 76,  40],
           [215,  76],
           [137, 105]], dtype=int32),
    codes=array([[[ 0.00075224, -0.00508452, -0.00209744, ..., -0.00410097,
              0.00487271,  0.01143065],
            [-0.01336251, -0.00854813, -0.01657207, ..., -0.0121797 ,
             -0.04402795,  0.09026501],
            [ 0.01800587, -0.00977852, -0.00773467, ...,  0.01342367,
             -0.00783198, -0.00703052],
            ...,
            [ 0.01005863,  0.00398296,  0.000291  , ..., -0.00291621,
              0.00306022, -0.02094289],
            [-0.00147054, -0.01458951, -0.01213949, ...,  0.00578341,
              0.00209298,  0.00941269],
            [ 0.03381471, -0.06974418, -0.01579198, ...,  0.00762085,
              0.00227993,  0.03332646]],
    
           [[ 0.02149276, -0.03359466,  0.0463151 , ...,  0.02963

In [11]:
from evaluation import vecs_similarity

q = vecs_similarity(ft, ft_quantitized, ["length", "sz"])
q

0.40992091278691123

In [24]:
from scipy.spatial import distance

distance.cosine(original_vec, new_ft.get_vector("sz"))

0.42782288789749146

In [8]:
# import pickle

# with open("out/original.bin", 'wb') as file:
#     pickle.dump(ft, file)

# ft.save("out/original")

# save_facebook_model(ft, "out/original.bin")

In [10]:
FastText.load("out/original")

IndexError: index 500000 is out of bounds for axis 0 with size 500000

In [75]:
new_ft = copy.deepcopy(ft)

In [76]:
new_ft.wv.vectors_ngrams.shape

(500000, 300)

In [77]:
from quantitizer import quantitize

new_ft.wv.vectors_ngrams = quantitize(new_ft.wv.vectors_ngrams, sub_size=2)
new_ft.wv.vectors = quantitize(new_ft.wv.vectors, sub_size=2)

KeyboardInterrupt: 

In [59]:
compressed_vec = new_ft.wv.get_vector("sz")

In [60]:
from scipy.spatial import distance

distance.cosine(original_vec, compressed_vec)

0

In [69]:
with open("out/compressed.bin", 'wb') as file:
    pickle.dump(new_ft, file)
# save_facebook_model(new_ft, "out/compressed.bin")

In [74]:
ft.wv.vectors.nbytes // 1024 // 1024

2288

In [73]:
ft.wv.vectors_ngrams.nbytes // 1024 // 1024

572

In [72]:
new_ft.wv.vectors_ngrams.nbytes // 1024 // 1024

4